In [25]:
import pandas as pd
import requests
import bs4 as bfs
import nltk
import syllables
from bs4 import BeautifulSoup
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import string
from textblob import TextBlob
import csv
import numpy as np

In [26]:
punkt_path = "D:/jupyter/TextAnalysisAssignment/punkt"


In [27]:
nltk.data.path.append(punkt_path)

In [28]:
input_df = pd.read_excel("Input.xlsx")
input_df

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...
...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...
97,blackassign0098,https://insights.blackcoffer.com/contribution-...
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...


In [29]:
li = [url for url in input_df['URL']]
li

['https://insights.blackcoffer.com/rising-it-cities-and-its-impact-on-the-economy-environment-infrastructure-and-city-life-by-the-year-2040-2/',
 'https://insights.blackcoffer.com/rising-it-cities-and-their-impact-on-the-economy-environment-infrastructure-and-city-life-in-future/',
 'https://insights.blackcoffer.com/internet-demands-evolution-communication-impact-and-2035s-alternative-pathways/',
 'https://insights.blackcoffer.com/rise-of-cybercrime-and-its-effect-in-upcoming-future/',
 'https://insights.blackcoffer.com/ott-platform-and-its-impact-on-the-entertainment-industry-in-future/',
 'https://insights.blackcoffer.com/the-rise-of-the-ott-platform-and-its-impact-on-the-entertainment-industry-by-2040/',
 'https://insights.blackcoffer.com/rise-of-cyber-crime-and-its-effects/',
 'https://insights.blackcoffer.com/rise-of-internet-demand-and-its-impact-on-communications-and-alternatives-by-the-year-2035-2/',
 'https://insights.blackcoffer.com/rise-of-cybercrime-and-its-effect-by-the-ye

In [30]:
nltk_data_path = "D:/jupyter/TextAnalysisAssignment/StopWords"
nltk.data.path.append(nltk_data_path)

In [31]:
def extract_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    article_div = soup.find('div', class_='article')
    if article_div:
        article_text = article_div.get_text(separator='\n')
        return article_text.strip()
    else:
        return "No article content found."

In [56]:
def load_word_list(file_path):
    with open(file_path, 'r') as file:
        word_list = [line.strip() for line in file]
    return word_list

positive_words = load_word_list('D:/jupyter/TextAnalysisAssignment/MasterDictionary/positive-words.txt')
negative_words = load_word_list('D:/jupyter/TextAnalysisAssignment/MasterDictionary/negative-words.txt')





def analyze_text(text,positive_words, negative_words):
    blob = TextBlob(text)
    positive_score = sum(word in positive_words for word in blob.words)
    negative_score = sum(word in negative_words for word in blob.words)
    polarity_score =  positive_score - negative_score
    subjectivity_score = blob.sentiment.subjectivity
    words = word_tokenize(text)
    sentences = sent_tokenize(text)
  
    avg_sentence_length = sum(len(sent.split()) for sent in sentences) / len(sentences)
    percentage_of_complex_words = sum(1 for word in words if syllables.estimate(word) > 2) / len(words) * 100
    fog_index = 0.4 * (avg_sentence_length + percentage_of_complex_words)
    avg_number_of_words_per_sentence = len(words) / len(sentences)
    complex_word_count = sum(1 for word in words if syllables.estimate(word) > 2)
    word_count = len(words)
    syllable_per_word = sum(syllables.estimate(word) for word in words) / word_count
    personal_pronouns = sum(1 for word in words if word.lower() in ['i', 'me', 'my', 'mine', 'myself'])
    avg_word_length = sum(len(word) for word in words) / word_count
    
    return {
        'POSITIVE SCORE': positive_score,
        'NEGATIVE SCORE': negative_score,
        'POLARITY SCORE': polarity_score,
        'SUBJECTIVITY SCORE': subjectivity_score,
        'AVG SENTENCE LENGTH': avg_sentence_length,
        'PERCENTAGE OF COMPLEX WORDS': percentage_of_complex_words,
        'FOG INDEX': fog_index,
        'AVG NUMBER OF WORDS PER SENTENCE': avg_number_of_words_per_sentence,
        'COMPLEX WORD COUNT': complex_word_count,
        'WORD COUNT': word_count,
        'SYLLABLE PER WORD': syllable_per_word,
        'PERSONAL PRONOUNS': personal_pronouns,
        'AVG WORD LENGTH': avg_word_length,
    }

In [34]:
output_rows = []
for index, row in input_df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    article_text = extract_text(url)
    analysis_results = analyze_text(article_text, positive_words, negative_words)
    row_dict = {'URL_ID': url_id, 'URL': url, **analysis_results}
    output_rows.append(row_dict)
output_df = pd.DataFrame(output_rows)


In [52]:
print(output_df)

             URL_ID                                                URL   
0   blackassign0001  https://insights.blackcoffer.com/rising-it-cit...  \
1   blackassign0002  https://insights.blackcoffer.com/rising-it-cit...   
2   blackassign0003  https://insights.blackcoffer.com/internet-dema...   
3   blackassign0004  https://insights.blackcoffer.com/rise-of-cyber...   
4   blackassign0005  https://insights.blackcoffer.com/ott-platform-...   
..              ...                                                ...   
95  blackassign0096  https://insights.blackcoffer.com/what-is-the-r...   
96  blackassign0097  https://insights.blackcoffer.com/impact-of-cov...   
97  blackassign0098  https://insights.blackcoffer.com/contribution-...   
98  blackassign0099  https://insights.blackcoffer.com/how-covid-19-...   
99  blackassign0100  https://insights.blackcoffer.com/how-will-covi...   

    POSITIVE SCORE  NEGATIVE SCORE  POLARITY SCORE  SUBJECTIVITY SCORE   
0                0               0   

In [36]:
output_df.to_excel("Analysis_Results.xlsx", index=True)